## Project setup

In [ ]:
pip install playwright jmespath scrapfly-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install nest_asyncio

In [ ]:
!playwright install

161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 7.5s161.3 MiB [] 0% 5.2s161.3 MiB [] 1% 3.7s161.3 MiB [] 1% 3.2s161.3 MiB [] 2% 3.0s161.3 MiB [] 3% 2.8s161.3 MiB [] 4% 2.6s161.3 MiB [] 4% 2.7s161.3 MiB [] 5% 2.6s161.3 MiB [] 5% 2.5s161.3 MiB [] 6% 2.5s161.3 MiB [] 6% 2.7s161.3 MiB [] 7% 2.8s161.3 MiB [] 8% 2.7s161.3 MiB [] 9% 2.6s161.3 MiB [] 9% 2.7s161.3 MiB [] 10% 2.6s161.3 MiB [] 11% 2.6s161.3 MiB [] 12% 2.5s161.3 MiB [] 13% 2.4s161.3 MiB [] 14% 2.3s161.3 MiB [] 14% 2.4s161.3 MiB [] 15% 2.4s161.3 MiB [] 15% 2.3s161.3 MiB [] 16% 2.2s161.3 MiB [] 17% 2.2s161.3 MiB [] 18% 2.1s161.3 MiB [] 19% 2.1s161.3 MiB [] 20% 2.0s161.3 MiB [] 21% 2.0s161.3 MiB [] 22% 1.9s161.3 MiB [] 23% 1.9s161.3 MiB [] 24% 1.9s161.3 MiB [] 25% 1.9s161.3 MiB [] 26% 1.8s161.3 MiB [] 27% 1.8s161.3 MiB [] 28% 1.7s161.3 MiB [] 29% 1.7s161.3 MiB [] 31% 1.6s161.3 MiB [] 32% 1.6s161.3 MiB [] 33% 1.5s161.3 MiB [] 34% 1.5s161.3 MiB [] 36% 1.4s161.3 MiB [] 37% 1.4s161.3 MiB [] 38% 1.3s161.3 MiB [] 39% 1.3s161.3 MiB [] 40% 1.3s161.3 MiB

## Scraping tweets

In [ ]:
from playwright.async_api import async_playwright
import asyncio
import nest_asyncio

async def scrape_tweet(url: str) -> dict:
    """
    Scrape a single tweet page for Tweet thread e.g.:
    https://twitter.com/Scrapfly_dev/status/1667013143904567296
    Return parent tweet, reply tweets and recommended tweets
    """
    _xhr_calls = []

    def intercept_response(response):
        """capture all background requests and save them"""
        # we can extract details from background requests
        if response.request.resource_type == "xhr":
            _xhr_calls.append(response)
        return response

    async with async_playwright() as pw: # Use async_playwright and async with
        browser = await pw.chromium.launch(headless=True) # Use await for asynchronous operations
        context = await browser.new_context(viewport={"width": 1920, "height": 1080}) # Use await for asynchronous operations
        page = await context.new_page() # Use await for asynchronous operations

        # enable background request intercepting:
        page.on("response", intercept_response)
        # go to url and wait for the page to load
        await page.goto(url) # Use await for asynchronous operations
        await page.wait_for_selector("[data-testid='tweet']") # Use await for asynchronous operations

        # find all tweet background requests:
        tweet_calls = [f for f in _xhr_calls if "TweetResultByRestId" in f.url]
        for xhr in tweet_calls:
            data = await xhr.json() # Use await for asynchronous operations
            return data['data']['tweetResult']['result']



# Instead of asyncio.run, get the current event loop using asyncio.get_event_loop
# and run the scrape_tweet coroutine using loop.run_until_complete
if __name__ == "__main__":
    # Apply nest_asyncio to allow nested event loops within Jupyter
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(scrape_tweet("https://twitter.com/Scrapfly_dev/status/1664267318053179398"))
    print(result)

{'__typename': 'Tweet', 'rest_id': '1664267318053179398', 'core': {'user_results': {'result': {'__typename': 'User', 'id': 'VXNlcjoxMzEwNjIzMDgxMzAwNDAyMTc4', 'rest_id': '1310623081300402178', 'affiliates_highlighted_label': {}, 'is_blue_verified': True, 'profile_image_shape': 'Circle', 'legacy': {'created_at': 'Mon Sep 28 16:51:22 +0000 2020', 'default_profile': True, 'default_profile_image': False, 'description': 'API products for developers:\n- Web Scraping API: scrape any page\n- Screenshot API: screenshot any website\n- Extraction API: parse data using AI', 'entities': {'description': {'urls': []}, 'url': {'urls': [{'display_url': 'scrapfly.io', 'expanded_url': 'https://scrapfly.io', 'url': 'https://t.co/1Is3k6KzyM', 'indices': [0, 23]}]}}, 'fast_followers_count': 0, 'favourites_count': 38, 'followers_count': 256, 'friends_count': 4, 'has_custom_timelines': True, 'is_translator': False, 'listed_count': 3, 'location': 'Paris', 'media_count': 38, 'name': 'Scrapfly', 'normal_follower

## Parsing tweet dataset

In [ ]:
from typing import Dict

def parse_tweet(data: Dict) -> Dict:
    """Parse Twitter tweet JSON dataset for the most important fields"""
    result = jmespath.search(
        """{
        created_at: legacy.created_at,
        attached_urls: legacy.entities.urls[].expanded_url,
        attached_urls2: legacy.entities.url.urls[].expanded_url,
        attached_media: legacy.entities.media[].media_url_https,
        tagged_users: legacy.entities.user_mentions[].screen_name,
        tagged_hashtags: legacy.entities.hashtags[].text,
        favorite_count: legacy.favorite_count,
        bookmark_count: legacy.bookmark_count,
        quote_count: legacy.quote_count,
        reply_count: legacy.reply_count,
        retweet_count: legacy.retweet_count,
        quote_count: legacy.quote_count,
        text: legacy.full_text,
        is_quote: legacy.is_quote_status,
        is_retweet: legacy.retweeted,
        language: legacy.lang,
        user_id: legacy.user_id_str,
        id: legacy.id_str,
        conversation_id: legacy.conversation_id_str,
        source: source,
        views: views.count
    }""",
        data,
    )
    result["poll"] = {}
    poll_data = jmespath.search("card.legacy.binding_values", data) or []
    for poll_entry in poll_data:
        key, value = poll_entry["key"], poll_entry["value"]
        if "choice" in key:
            result["poll"][key] = value["string_value"]
        elif "end_datetime" in key:
            result["poll"]["end"] = value["string_value"]
        elif "last_updated_datetime" in key:
            result["poll"]["updated"] = value["string_value"]
        elif "counts_are_final" in key:
            result["poll"]["ended"] = value["boolean_value"]
        elif "duration_minutes" in key:
            result["poll"]["duration"] = value["string_value"]
    user_data = jmespath.search("core.user_results.result", data)
    if user_data:
        result["user"] = parse_user(user_data)
    return result

SyntaxError: invalid decimal literal (<ipython-input-1-15c19a8cb294>, line 1)